# Cuaderno de Segmentación y Análisis Calibrado de Astrocitos

**Objetivo:** Este notebook detalla un flujo de trabajo para identificar núcleos de astrocitos, guardando y visualizando cada paso de filtrado. El análisis y la visualización están **calibrados con las dimensiones físicas del vóxel** para asegurar la precisión y reproducibilidad científica.

**Flujo de Trabajo:**
1.  **Configuración**: Definición de rutas y parámetros de calibración física y procesamiento.
2.  **Carga de Datos**: Carga de la imagen original (DAPI, GFAP y Microglía).
3.  **Pre-procesamiento (Otsu)**: Limpieza del canal DAPI.
4.  **Segmentación (Cellpose)**: Segmentación de todos los núcleos.
5.  **Filtrado Combinado**: Selección de núcleos por señal GFAP y exclusión por proximidad a señal de Microglía.
6.  **Post-procesamiento (Tamaño Físico)**: Limpieza final por volumen físico (µm³).
7.  **Visualización Integrada**: Visualización de todas las máscaras intermedias y finales en Napari.

## Paso 0: Activar el Backend Gráfico

**Importante:** Ejecuta esta celda una sola vez por sesión. El comando mágico `%gui qt` prepara el notebook para mostrar ventanas interactivas como las de Napari.

In [1]:
%gui qt

## Paso 1: Configuración de Rutas y Parámetros

Esta celda centraliza todas las variables que controlan el flujo de trabajo. Definir los parámetros aquí permite ajustar fácilmente el análisis para diferentes imágenes sin modificar el código en las celdas posteriores.

### Fundamento del Filtrado y Reproducibilidad Científica 🔬

La estrategia de este cuaderno se basa en un **filtrado secuencial**, un método robusto y común en el análisis de imágenes biológicas. En lugar de depender de un único algoritmo "mágico", aplicamos una serie de filtros lógicos, cada uno diseñado para eliminar un tipo específico de artefacto o señal no deseada.

Este enfoque mejora la **reproducibilidad científica** por varias razones:
1.  **Transparencia**: Cada paso del filtrado (Otsu, Cellpose, Co-localización, Tamaño) es explícito y sus parámetros están claramente definidos.
2.  **Objetividad**: Al definir umbrales y parámetros numéricos, reducimos la subjetividad inherente a la selección manual.
3.  **Adaptabilidad**: Si se utiliza un nuevo set de imágenes, solo es necesario reajustar los parámetros en esta celda para adaptar el mismo flujo de trabajo lógico, manteniendo la consistencia del método.
4.  **Calibración**: En el Paso 2, la calibración física (tamaño de vóxel en X, Y, Z) se lee automáticamente desde los metadatos OME-XML del archivo `.tif` utilizando `tifffile`. A partir de esos valores calculamos `PHYSICAL_SCALE` y `VOXEL_VOLUME_UM3` y convertimos `MIN_VOLUME_UM3` a `MIN_VOLUME_VOXELS` sin intervención manual.

---

### Variables de Parámetros de Procesamiento

* **`NUCLEUS_DIAMETER`**:
    * **Uso**: En la **Celda de Segmentación con Cellpose**.
    * **Explicación**: Es el parámetro más importante para Cellpose. Le informa al modelo sobre el tamaño esperado (en píxeles) de los objetos que debe buscar. Un valor correcto mejora drásticamente la precisión de la segmentación.

* **`MAX_DILATION_ITERATIONS`**:
    * **Uso**: En la **Celda de Filtrado Combinado**.
    * **Explicación**: Controla la **distancia máxima de búsqueda** para la señal de GFAP o microglía alrededor de cada núcleo. El script expandirá un anillo hasta este número de iteraciones buscando una señal significativa.

* **`GFAP_INTENSITY_THRESHOLD`**:
    * **Uso**: En la **Celda de Filtrado Combinado**.
    * **Explicación**: Es el umbral de decisión para considerar que una señal GFAP es positiva. Si la intensidad media en el anillo de búsqueda supera este valor (y se encuentra antes que la señal de microglía), el núcleo es aceptado.

* **`MICROGLIA_INTENSITY_THRESHOLD`** (Nuevo):
    * **Uso**: En la **Celda de Filtrado Combinado**.
    * **Explicación**: Es el umbral de decisión para la exclusión. Si la intensidad media de la microglía en el anillo de búsqueda supera este valor antes que la señal de GFAP, el núcleo se descarta.

* **`MIN_VOLUME_UM3`**:
    * **Uso**: En la **Celda de Post-procesamiento por Tamaño**.
    * **Explicación**: Es el parámetro para el filtro de limpieza final, **definido en unidades físicas (micrómetros cúbicos, µm³)**. El script lo convierte automáticamente a un umbral en vóxeles (`MIN_VOLUME_VOXELS`) usando las dimensiones de calibración leídas del OME-XML.

In [ ]:
from pathlib import Path


# --- Rutas de Archivos ---
project_root = Path.cwd().parent
base_filename = "Inmuno 26-07-23.lif - CTL 1-2 a"
subfolder = "CTL/CTL 1-2"
image_path = project_root / f"data/raw/{subfolder}/{base_filename}.tif"


# --- Rutas de Salida ---
output_dir = project_root / "data/processed" / base_filename
output_dir.mkdir(parents=True, exist_ok=True)
otsu_mask_path = output_dir / "01_otsu_mask.tif"
cellpose_mask_path = output_dir / "02_cellpose_mask.tif"
gfap_filtered_mask_path = output_dir / "03_gfap_microglia_filtered_mask.tif"
final_mask_path = output_dir / "04_final_astrocytes_mask.tif"


# --- Archivo opcional de overrides de calibración (si no hay metadatos en el TIF) ---
calibration_overrides_path = project_root / "data/calibration_overrides.json"  # (opcional, legado)
streamlit_calibration_path = project_root / "streamlit/calibration.json"  # calibración global preferida


# --- Parámetros de Procesamiento ---
# NOTA: Los parámetros de calibración física se leerán automáticamente del archivo de imagen.
NUCLEUS_DIAMETER = 30
MAX_DILATION_ITERATIONS = 20
GFAP_INTENSITY_THRESHOLD = 10
MICROGLIA_INTENSITY_THRESHOLD = 150
MIN_VOLUME_UM3 = 75


# La conversión de MIN_VOLUME_UM3 a vóxeles se hará después de leer la calibración.
print("Parámetros de procesamiento definidos. La calibración física se obtendrá de metadatos del TIF o del override JSON si existe.")

Parámetros de procesamiento definidos. La calibración física se obtendrá de metadatos del TIF o del override JSON si existe.


In [12]:
# Utilidades para leer metadatos OME-XML de forma robusta con manejo de unidades


def _to_um(value: float, unit: str | None) -> float | None:

    """Convierte un valor de longitud a micrómetros (µm) usando una unidad OME común."""

    if value is None:

        return None

    if not unit:

        return float(value)

    unit = unit.strip().lower()

    # Mapeo básico de unidades a µm

    if unit in {"µm", "um", "micrometer", "micrometre"}:

        return float(value)

    if unit in {"nm", "nanometer", "nanometre"}:

        return float(value) / 1000.0

    if unit in {"mm", "millimeter", "millimetre"}:

        return float(value) * 1000.0

    if unit in {"m", "meter", "metre"}:

        return float(value) * 1_000_000.0

    # Desconocida: devolvemos el valor tal cual, asumiendo µm (mejor avisar arriba si se desea)

    return float(value)



def extract_physical_sizes_from_ome_xml(ome_xml: str):


    """


    Extrae (PhysicalSizeZ, PhysicalSizeY, PhysicalSizeX) en µm desde OME-XML.

    Intenta varios espacios de nombres y usa unidades si están presentes.

    Devuelve tupla (z_um, y_um, x_um) con None si no se encuentra.

    """

    if not ome_xml:

        return (None, None, None)



    import xml.etree.ElementTree as _ET



    namespaces = [

        "http://www.openmicroscopy.org/Schemas/OME/2016-06",

        "http://www.openmicroscopy.org/Schemas/OME/2015-01",

        "http://www.openmicroscopy.org/Schemas/OME/2013-06",

    ]



    try:

        root = _ET.fromstring(ome_xml)

    except Exception:

        return (None, None, None)



    def _read_from_pixels(p):

        z = p.attrib.get("PhysicalSizeZ")

        y = p.attrib.get("PhysicalSizeY")

        x = p.attrib.get("PhysicalSizeX")

        zu = p.attrib.get("PhysicalSizeZUnit") or p.attrib.get("PhysicalSizeZUnitSymbol")

        yu = p.attrib.get("PhysicalSizeYUnit") or p.attrib.get("PhysicalSizeYUnitSymbol")

        xu = p.attrib.get("PhysicalSizeXUnit") or p.attrib.get("PhysicalSizeXUnitSymbol")

        z_um = _to_um(float(z), zu) if z is not None else None

        y_um = _to_um(float(y), yu) if y is not None else None

        x_um = _to_um(float(x), xu) if x is not None else None

        return (z_um, y_um, x_um)



    # 1) Namespaces conocidos

    for ns in namespaces:

        pixels = root.find(f".//{{{ns}}}Pixels")

        if pixels is not None:

            return _read_from_pixels(pixels)



    # 2) Fallback sin namespace

    pixels = root.find('.//Pixels')

    if pixels is not None:

        return _read_from_pixels(pixels)



    return (None, None, None)



def extract_sizes_from_ome_xml(ome_xml: str):


    """Extrae SizeZ, SizeC, SizeY, SizeX desde OME-XML si están presentes."""

    if not ome_xml:

        return {}

    import xml.etree.ElementTree as _ET

    namespaces = [

        "http://www.openmicroscopy.org/Schemas/OME/2016-06",

        "http://www.openmicroscopy.org/Schemas/OME/2015-01",

        "http://www.openmicroscopy.org/Schemas/OME/2013-06",

    ]

    try:

        root = _ET.fromstring(ome_xml)

    except Exception:

        return {}

    def _sizes_from_pixels(p):

        out = {}

        for k in ("SizeZ","SizeC","SizeY","SizeX","SizeT"):

            v = p.attrib.get(k)

            if v is not None:

                try:

                    out[k] = int(v)

                except Exception:

                    pass

        # DimensionOrder si está

        do = p.attrib.get("DimensionOrder")

        if do:

            out["DimensionOrder"] = do

        return out

    for ns in namespaces:

        pixels = root.find(f".//{{{ns}}}Pixels")

        if pixels is not None:

            sizes = _sizes_from_pixels(pixels)

            if sizes:

                return sizes

    pixels = root.find('.//Pixels')

    if pixels is not None:

        return _sizes_from_pixels(pixels)

    return {}

## Paso 2: Carga de Datos y Calibración Automática

Utilizamos `tifffile` para abrir el archivo y leer tanto los datos como los metadatos OME-XML. A partir de esos metadatos extraemos las dimensiones físicas del vóxel (X, Y, Z) en micrómetros. Una vez leídos, calculamos los parámetros de calibración y procedemos a separar los canales.

In [ ]:
import json


# ... (código anterior de lectura de TIF y reordenación) se mantiene arriba


# 1) OME-XML (preferido)
Z_SPACING_UM, PIXEL_HEIGHT_UM, PIXEL_WIDTH_UM = extract_physical_sizes_from_ome_xml(OME_XML)
calib_source = None
if Z_SPACING_UM or PIXEL_HEIGHT_UM or PIXEL_WIDTH_UM:
    calib_source = "OME-XML"


# 1.3) Override JSON global de Streamlit (si existe) — preferido
try:
    if 'streamlit_calibration_path' in globals() and streamlit_calibration_path.exists():
        with open(streamlit_calibration_path, 'r') as f:
            ov_glob = json.load(f)
        z = ov_glob.get('z')
        y = ov_glob.get('y')
        x = ov_glob.get('x')
        if z is not None:
            Z_SPACING_UM = float(z)
        if y is not None:
            PIXEL_HEIGHT_UM = float(y)
        if x is not None:
            PIXEL_WIDTH_UM = float(x)
        calib_source = 'Streamlit-JSON'
except Exception as e:
    print(f"Advertencia: No se pudo leer streamlit/calibration.json: {e}")
# 1.5) Override JSON por nombre de archivo (si existe) — compatibilidad
try:
    if 'calibration_overrides_path' in globals() and calibration_overrides_path.exists():
        with open(calibration_overrides_path, 'r') as f:
            overrides = json.load(f)
        ov = overrides.get(base_filename) or overrides.get(image_path.name)
        if ov:
            z = ov.get('z')
            y = ov.get('y')
            x = ov.get('x')
            if z is not None:
                Z_SPACING_UM = float(z)
            if y is not None:
                PIXEL_HEIGHT_UM = float(y)
            if x is not None:
                PIXEL_WIDTH_UM = float(x)
            calib_source = "Override-JSON" if (z or y or x) else calib_source
except Exception as e:
    print(f"Advertencia: No se pudo leer calibration_overrides.json: {e}")


# 2) Fallback ImageJ metadata
if calib_source is None and IJ_META:
    try:
        unit = IJ_META.get('unit')  # p.ej., 'micron', 'um'
        spacing = IJ_META.get('spacing')  # distancia entre slices en 'unit'
        z_um = _to_um(float(spacing), unit) if spacing is not None else None
        if z_um is not None:
            Z_SPACING_UM = Z_SPACING_UM or z_um
            calib_source = "ImageJ"
    except Exception:
        pass


# 3) Fallback a tags XResolution/YResolution (+ ResolutionUnit)
if (PIXEL_WIDTH_UM is None or PIXEL_HEIGHT_UM is None) and XRES and RES_UNIT:
    if RES_UNIT == 2:  # inch
        PIXEL_WIDTH_UM = PIXEL_WIDTH_UM or (25400.0 / XRES)
        if YRES:
            PIXEL_HEIGHT_UM = PIXEL_HEIGHT_UM or (25400.0 / YRES)
        calib_source = calib_source or "TIFF-Resolution-Inch"
    elif RES_UNIT == 3:  # centimeter
        PIXEL_WIDTH_UM = PIXEL_WIDTH_UM or (10000.0 / XRES)
        if YRES:
            PIXEL_HEIGHT_UM = PIXEL_HEIGHT_UM or (10000.0 / YRES)
        calib_source = calib_source or "TIFF-Resolution-Cm"


# Fallbacks finales si aún faltan
if Z_SPACING_UM is None:
    Z_SPACING_UM = 1.0
if PIXEL_HEIGHT_UM is None:
    PIXEL_HEIGHT_UM = 1.0
if PIXEL_WIDTH_UM is None:
    PIXEL_WIDTH_UM = 1.0
calib_source = calib_source or "Default-1um"

In [14]:
# Chequeo rápido de forma y ejes
print("Forma del arreglo (esperado 4D Z,C,Y,X):", image_stack.shape)

sizes = extract_sizes_from_ome_xml(OME_XML) if 'OME_XML' in globals() else {}
print("Sizes OME (si disponibles):", sizes)

if image_stack.ndim == 4 and image_stack.shape[1] not in (3, 4):
    print("ADVERTENCIA: el eje 1 no parece ser el de canales (tamaño 3/4 típico). Revisa la reordenación de ejes.")

Forma del arreglo (esperado 4D Z,C,Y,X): (11, 3, 1024, 1024)
Sizes OME (si disponibles): {}


## Paso 3: Pre-procesamiento con Umbral de Otsu
Limpiamos el canal DAPI para eliminar ruido de fondo y guardamos el resultado.

In [15]:
print("Aplicando umbral de Otsu...")
otsu_threshold = threshold_otsu(dapi_channel)
otsu_mask = dapi_channel > otsu_threshold
dapi_channel_cleaned = np.where(otsu_mask, dapi_channel, 0)

tifffile.imwrite(otsu_mask_path, otsu_mask.astype(np.uint8))
print(f"Máscara de Otsu guardada en: {otsu_mask_path}")

Aplicando umbral de Otsu...
Máscara de Otsu guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/01_otsu_mask.tif


### Visualización: Máscara de Otsu (Calibrada)

In [16]:
viewer_otsu = napari.Viewer()
viewer_otsu.add_image(dapi_channel, name='DAPI Original', colormap='blue', scale=PHYSICAL_SCALE)
viewer_otsu.add_image(microglia_channel, name='Microglía Original', colormap='red', scale=PHYSICAL_SCALE)
viewer_otsu.add_image(gfap_channel, name='GFAP Original', colormap='green', scale=PHYSICAL_SCALE)
viewer_otsu.add_labels(otsu_mask, name='Máscara de Otsu', scale=PHYSICAL_SCALE)

/home/daniel/Proyectos/astrocitos-3d-analysis/venv/lib/python3.12/site-packages/npe2/manifest/_npe1_adapter.py:87: UserWarning: Error importing contributions for first-generation napari plugin 'bfio': partially initialized module 'bfio' has no attribute 'base_classes' (most likely due to a circular import)
  self._load_contributions()


<Labels layer 'Máscara de Otsu' at 0x7620d4375100>

## Paso 4: Segmentación de Núcleos con Cellpose
Ejecutamos Cellpose sobre el canal DAPI limpio y guardamos la máscara de etiquetas resultante.

In [8]:
model = models.CellposeModel(gpu=True)
print("Ejecuting segmentación con Cellpose...")
cellpose_masks, _, _ = model.eval(
    dapi_channel_cleaned,
    diameter=NUCLEUS_DIAMETER,
    z_axis=0,
    do_3D=True
)

tifffile.imwrite(cellpose_mask_path, cellpose_masks.astype(np.uint16))
print(f"Máscara de Cellpose guardada en: {cellpose_mask_path}")

if cellpose_masks.max() > 0:
    props = regionprops_table(cellpose_masks, properties=('label',))
    print(f"\nNúmero total de núcleos encontrados por Cellpose: {len(props['label'])}")
else:
    print("\nNo se encontraron objetos en la segmentación inicial.")

Ejecuting segmentación con Cellpose...
Máscara de Cellpose guardada en: /home/imagina/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/02_cellpose_mask.tif

Número total de núcleos encontrados por Cellpose: 362


### Visualización: Resultado de Cellpose (Calibrada)

In [29]:
viewer_cellpose = napari.Viewer()
viewer_cellpose.add_image(dapi_channel, name='DAPI Original', colormap='blue', scale=PHYSICAL_SCALE)
viewer_cellpose.add_labels(cellpose_masks, name='Resultado Cellpose (Crudo)', scale=PHYSICAL_SCALE)

<Labels layer 'Resultado Cellpose (Crudo)' at 0x7b849d466c90>

## Paso 5: Filtrado Combinado por Co-localización GFAP y Exclusión por Microglía

Este es el paso de filtrado principal, ahora con la lógica actualizada. Para cada núcleo, expandimos un anillo a su alrededor, píxel por píxel, y comprobamos qué señal (GFAP o Microglía) supera primero su umbral.

- **Éxito (Candidato a Astrocito)**: Si la señal GFAP es la primera en ser detectada.
- **Fallo (Núcleo Excluido)**: Si la señal de Microglía es detectada primero o al mismo tiempo.

In [27]:
import numpy as np
import tifffile
from skimage.measure import regionprops
from skimage.morphology import binary_dilation
import concurrent.futures # <-- Importamos la librería necesaria

# --- Constantes (sin cambios) ---
MAX_DILATION_ITERATIONS = 10
GFAP_INTENSITY_THRESHOLD = 40
MICROGLIA_INTENSITY_THRESHOLD = 200

# --- 1. Lógica de procesamiento encapsulada en una función ---
# Esta función contiene la lógica para procesar un ÚNICO núcleo.
# Devolverá el 'label' del núcleo si es un candidato, o None si es descartado.
def process_nucleus(nucleus, cellpose_masks, gfap_channel, microglia_channel):
    """
    Analiza un único núcleo para determinar si es un candidato a astrocito.
    """
    nucleus_mask = (cellpose_masks == nucleus.label)
    current_mask = nucleus_mask
    
    for _ in range(MAX_DILATION_ITERATIONS):
        dilated_mask = binary_dilation(current_mask)
        # El 'shell' es el borde de la dilatación, para no re-analizar píxeles
        shell_mask = dilated_mask & ~current_mask
        
        # Si la dilatación no crece más, paramos
        if not np.any(shell_mask):
            break
            
        shell_gfap_intensity = gfap_channel[shell_mask].mean()
        shell_microglia_intensity = microglia_channel[shell_mask].mean()
        
        # Criterio de exclusión: si encontramos señal de microglía, descartamos el núcleo
        if shell_microglia_intensity > MICROGLIA_INTENSITY_THRESHOLD:
            return None # Retornamos None para indicar que no es un candidato
            
        # Criterio de inclusión: si encontramos suficiente señal de GFAP, lo aceptamos
        if shell_gfap_intensity > GFAP_INTENSITY_THRESHOLD:
            return nucleus.label # Retornamos el label del núcleo candidato
        
        # Actualizamos la máscara para la siguiente iteración de dilatación
        current_mask = dilated_mask
    
    # Si el bucle termina sin cumplir el criterio de GFAP, no es un candidato
    return None

# --- Bloque principal de ejecución ---

# Asumo que las variables cellpose_masks, gfap_channel, y microglia_channel ya están cargadas
# Ejemplo de carga (descomentar y adaptar si es necesario):
# cellpose_masks = tifffile.imread('path/to/your/cellpose_masks.tif')
# gfap_channel = tifffile.imread('path/to/your/gfap_channel.tif')
# microglia_channel = tifffile.imread('path/to/your/microglia_channel.tif')
# gfap_filtered_mask_path = 'path/to/your/output_mask.tif'


print("Iniciando filtrado combinado por señal GFAP y exclusión por Microglía (en paralelo)...")
nuclei_props = regionprops(cellpose_masks)

# Lista para guardar los resultados finales
astrocyte_labels_candidate = []

# --- 2. Usamos ProcessPoolExecutor para paralelizar ---
# 'with' se asegura de que el pool de procesos se cierre correctamente.
# Por defecto, usará tantos procesos como núcleos tenga tu CPU.
with concurrent.futures.ProcessPoolExecutor() as executor:
    
    # --- 3. Mapeamos la función a los datos ---
    # Creamos un "futuro" para cada tarea. Pasamos los argumentos necesarios a process_nucleus.
    # Como los arrays (masks, channels) son los mismos para todas las tareas, los pasamos en cada llamada.
    future_to_nucleus = {
        executor.submit(process_nucleus, nucleus, cellpose_masks, gfap_channel, microglia_channel): nucleus
        for nucleus in nuclei_props
    }
    
    # --- 4. Recolectamos los resultados a medida que se completan ---
    for future in concurrent.futures.as_completed(future_to_nucleus):
        result = future.result()
        # Si el resultado no es None, es una etiqueta válida
        if result is not None:
            astrocyte_labels_candidate.append(result)

# El resto del código es igual
gfap_filtered_mask = np.where(np.isin(cellpose_masks, astrocyte_labels_candidate), cellpose_masks, 0)

tifffile.imwrite(gfap_filtered_mask_path, gfap_filtered_mask.astype(np.uint16))
print(f"Máscara filtrada guardada en: {gfap_filtered_mask_path}")
print(f"\nProceso completado. Quedan {len(astrocyte_labels_candidate)} candidatos a astrocitos después del filtro combinado.")

Iniciando filtrado combinado por señal GFAP y exclusión por Microglía (en paralelo)...
Máscara filtrada guardada en: /home/imagina/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/03_gfap_microglia_filtered_mask.tif

Proceso completado. Quedan 54 candidatos a astrocitos después del filtro combinado.


### Visualización: Candidatos a Astrocitos (Calibrada)

In [28]:
viewer_gfap = napari.Viewer()
viewer_gfap.add_image(gfap_channel, name='GFAP', colormap='green', scale=PHYSICAL_SCALE)
viewer_gfap.add_image(microglia_channel, name='Microglia', colormap='magenta', scale=PHYSICAL_SCALE)
viewer_gfap.add_labels(gfap_filtered_mask, name='Candidatos a Astrocitos', scale=PHYSICAL_SCALE)

<Labels layer 'Candidatos a Astrocitos' at 0x7b84a0e318e0>

## Paso 6: Post-procesamiento por Tamaño Físico
Aplicamos el filtro final de limpieza, eliminando objetos con un volumen físico (µm³) menor al umbral definido.

In [30]:
print("Aplicando filtro final de tamaño...")
final_props = regionprops(gfap_filtered_mask)
final_astrocyte_labels = [prop.label for prop in final_props if prop.area >= MIN_VOLUME_VOXELS]

final_mask = np.where(np.isin(gfap_filtered_mask, final_astrocyte_labels), gfap_filtered_mask, 0)

tifffile.imwrite(final_mask_path, final_mask.astype(np.uint16))
print(f"Máscara final guardada en: {final_mask_path}")
print(f"Número final de astrocitos identificados: {len(final_astrocyte_labels)}")

Aplicando filtro final de tamaño...
Máscara final guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/04_final_astrocytes_mask.tif
Número final de astrocitos identificados: 169


## Paso 7: Visualización Final Integrada
Visualizamos la imagen original multi-canal junto a todas las máscaras generadas en el pipeline, permitiendo una revisión completa del proceso. Cada máscara se puede activar o desactivar para comparar los resultados de cada paso.

In [12]:
print("Abriendo Napari con todas las capas del pipeline...")

# Cargamos todas las máscaras guardadas para asegurar que tenemos las versiones correctas
otsu_mask_viz = tifffile.imread(otsu_mask_path)
cellpose_mask_viz = tifffile.imread(cellpose_mask_path)
gfap_filtered_mask_viz = tifffile.imread(gfap_filtered_mask_path)
final_mask_viz = tifffile.imread(final_mask_path)

viewer_final = napari.Viewer(title="Pipeline de Segmentación de Astrocitos")

# Capas de las señales originales
viewer_final.add_image(
    image_stack,
    channel_axis=1,
    name=["DAPI", "GFAP", "Microglia"],
    colormap=["blue", "green", "magenta"],
    scale=PHYSICAL_SCALE
)

# Capas de las máscaras generadas
viewer_final.add_labels(otsu_mask_viz, name='01 - Máscara Otsu', scale=PHYSICAL_SCALE, visible=False)
viewer_final.add_labels(cellpose_mask_viz, name='02 - Máscara Cellpose', scale=PHYSICAL_SCALE, visible=False)
viewer_final.add_labels(gfap_filtered_mask_viz, name='03 - Filtro GFAP y Microglia', scale=PHYSICAL_SCALE, visible=False)
viewer_final.add_labels(final_mask_viz, name='04 - Astrocitos Finales', scale=PHYSICAL_SCALE, visible=True)

Abriendo Napari con todas las capas del pipeline...


<Labels layer '04 - Astrocitos Finales' at 0x7b834c21a4e0>